# Invoking endpoint using lambda and Rest api


## API - [SageMaker Hugging Face Inference Toolkit](https://github.com/aws/sagemaker-huggingface-inference-toolkit)


Using the `transformers pipelines`, your inputs need to be defined in the `inputs` key and if you want additional supported `pipelines` parameters you can add them in the `parameters` key. Below you can find examples for requests. 

**text-classification request body**
```python
{
	"inputs": "Camera - You are awarded a SiPix Digital Camera! call 09061221066 fromm landline. Delivery within 28 days."
}
```
**question-answering request body**
```python
{
	"inputs": {
		"question": "What is used for inference?",
		"context": "My Name is Philipp and I live in Nuremberg. This model is used with sagemaker for inference."
	}
}
```
**zero-shot classification request body**
```python
{
	"inputs": "Hi, I recently bought a device from your company but it is not working as advertised and I would like to get reimbursed!",
	"parameters": {
		"candidate_labels": [
			"refund",
			"legal",
			"faq"
		]
	}
}
```

In [ ]:
!pip install "sagemaker>=2.48.0" --upgrade

## Deploy one of the 10 000+ Hugging Face Transformers to Amazon SageMaker for Inference


To deploy a model directly from the Hub to SageMaker we need to define 2 environment variables when creating the `HuggingFaceModel` . We need to define:

- `HF_MODEL_ID`: defines the model id, which will be automatically loaded from [huggingface.co/models](http://huggingface.co/models) when creating or SageMaker Endpoint. The 🤗 Hub provides +10 000 models all available through this environment variable.
- `HF_TASK`: defines the task for the used 🤗 Transformers pipeline. A full list of tasks can be find [here](https://huggingface.co/transformers/main_classes/pipelines.html).

In [1]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker 

role = sagemaker.get_execution_role()

# Hub Model configuration. https://huggingface.co/models
hub = {
  'HF_MODEL_ID':'distilbert-base-uncased-finetuned-sst-2-english', # model_id from hf.co/models
  'HF_TASK':'text-classification' # NLP task you want to use for predictions
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.6", # transformers version used
   pytorch_version="1.7", # pytorch version used
   py_version="py36", # python version of the DLC
)

In [2]:
huggingface_model.deploy??

In [3]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge",
)

-------------!

In [26]:
# example request, you always need to define "inputs"
# You can use two statements 
data = {
"inputs":  "he is a good person"
    
    #     if you have more than one input
    #     "inputs":  ("he is a good person","How are you")
    
}

# request
predictor.predict(data)

[{'label': 'POSITIVE', 'score': 0.9998661875724792}]

In [10]:
print(predictor.endpoint_name)

huggingface-pytorch-inference-2021-08-04-05-25-37-170


In [12]:
# Invoking endpoint with some data | use this in lambda
import boto3
import json
runtime = boto3.Session().client(service_name='runtime.sagemaker')

In [27]:
payload=data

In [40]:
response = runtime.invoke_endpoint(EndpointName="huggingface-pytorch-inference-2021-08-04-05-25-37-170",
ContentType='application/json',
Accept='application/json',
Body=bytes(json.dumps(payload),encoding='utf8'))

In [41]:
print(response)

{'ResponseMetadata': {'RequestId': 'fff1cc60-686a-4425-9849-d59386962f37',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fff1cc60-686a-4425-9849-d59386962f37',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Wed, 04 Aug 2021 07:21:53 GMT',
   'content-type': 'application/json',
   'content-length': '49'},
  'RetryAttempts': 0},
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7fefe29ee438>}

In [42]:
result = json.loads(response['Body'].read().decode())
print(result)

[{'label': 'POSITIVE', 'score': 0.9998661875724792}]

In [ ]:
# delete endpoint
predictor.delete_endpoint()